In [47]:
import json
import tweepy
import pandas as pd
import random

In [48]:
data=pd.read_csv("../../data/00-raw-data/rumor_manually.csv",encoding='windows-1252')
data=data.loc[:,"text":]
data.drop_duplicates(subset='twitter_id', keep='first',inplace=True)


In [49]:
# load key
consumer_key = "OA4CTZkyLwMOg08PlFt0goCNC"
consumer_secret = "8k59TeAGeqlHXjf4wjOjrKLYXpEvutuRgDjLxVxm5R7uC5wB0q"
access_token = "1177285260242874368-QoMZGUj2TsbkXyPQ5o0pu5iFrFT72q"
access_token_secret = "6JWjo6Ro4RmqIBFLjOop9QW1FCgHZltRTz8EIsj9UiMOE"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAISeXgEAAAAANwEgP0gtkLS%2Bjv9qCa3pIB3o1xs%3D98KYFXJI2DYHslg3StyrI2h9aB0w8PYq8iJyGS6Vdwsrhgkt7r"

#get authorization
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api=tweepy.API(auth)

In [50]:
rumor_source=data.twitter_id.tolist()# transform np.series to list so that we can use these id to look up their followers with tweepy
rumor_source

[105631171,
 112684779,
 739187198495477760,
 706551827672621058,
 2805007468,
 259001548,
 42278624,
 1556388209671737346,
 767788995962998784,
 5514192,
 255986116,
 2459542196,
 8940342,
 2525149488,
 3094974153,
 806344552718045184,
 18342379,
 39344374,
 1536373979790835713,
 926164634570067968,
 1447493161614553089,
 32300682,
 1355604149619077121,
 1467566088,
 149346219,
 1354582871223185408,
 977639698960191489,
 754718787237281792,
 878284831]

In [51]:
#definee a function to look up the followers
def followers_info(user_id_list):
    df=pd.DataFrame(columns=["user_id","screen_name","followers_count","friends_count","listed_count","favourites_count","tweet_num","protected","verfied","description","location"])
    for user in user_id_list:
        followers=api.get_follower_ids(user_id=user)
        if len(followers)>50:
            followers=random.sample(followers,50)
        df_merge=pd.DataFrame({"user_id":followers})
        respond=api.lookup_users(user_id=followers)
           # get the informatino from the structured data and store what we need in a list 
        location=[]
        followers_count=[]
        friends_count=[]
        description=[]
        listed_count=[]#The number of public lists that this user is a member of.
        favourites_count=[] #The number of Tweets this user has liked in the account’s lifetime. 
        tweet_num=[] #(statuses_count),The number of Tweets (including retweets) issued by the user.
        protected=[]#When true, indicates that this user has chosen to protect their Tweets.
        verified=[]#When true, indicates that the user has a verified account
        screen_name=[]
        
        for item in respond:
            screen_name.append(item._json["screen_name"])
            location.append(item._json["location"])
            followers_count.append(item._json["followers_count"])
            friends_count.append(item._json["friends_count"])
            description.append(item._json["description"])
            listed_count.append(item._json["listed_count"])
            favourites_count.append(item._json["favourites_count"])
            tweet_num.append(item._json["statuses_count"])
            protected.append(item._json["protected"])
            verified.append(item._json["verified"])


        #create new colums
        df_merge["screen_name"]=screen_name
        df_merge["followers_count"]=followers_count
        df_merge["friends_count"]=friends_count
        df_merge["listed_count"]=listed_count
        df_merge["favourites_count"]=favourites_count
        df_merge["tweet_num"]=tweet_num
        df_merge["protected"]=protected
        df_merge["verfied"]=verified
        df_merge["description"]=description
        df_merge["location"]=location
        df=pd.concat([df,df_merge])
    return(df)

In [52]:
#look up followers of rumor source and truth source
# due to the limitation of twitter, we can only use the "get_follower_id" function 15 times in one period.
rumor_followers=followers_info(random.sample(rumor_source,7))
truth_followers=followers_info([5402612,15012486,1120655269,380648579,86141342,7309052,15679641])
#label different groups
rumor_followers["label"]=1
truth_followers["label"]=0
followers=pd.concat([rumor_followers,truth_followers])
followers

,user_id,screen_name,followers_count,friends_count,listed_count,favourites_count,tweet_num,protected,verfied,description,location,label
0,2198516225,_Banzi_,27,199,0,601,122,False,False,,Joburg,1
1,1504258025804210176,DelorbeTori,81,2771,0,0,6,False,False,"Tori , 22😏 , Submissive🌸",,1
2,1581474148571877377,omkarVyawahare2,0,56,0,24,0,False,False,,,1
3,2967681610,L1BERTE_S,50,68,1,8735,2238,False,False,"Les querelles ne dureraient pas longtemps, si ...",Provence-Alpes-Côte d'Azur,1
4,1400890434,mistamomo_,356,1104,19,18347,20160,False,False,Amana.....abin riko da hanun dama 🤝 #freepales...,N/a,1
...,...,...,...,...,...,...,...,...,...,...,...,...
45,1576506437823504389,FranciscoPalob3,0,2,0,0,0,False,False,,,0
46,1570254264496177153,Nkindianfoods22,1,21,0,2,6,False,False,I m Narinder kaur I live in fort worth tx I ma...,"Fort Worth, TX",0
47,1580761571457994754,Dearl88447683,14,274,0,3,1,False,False,"Single, gay, goodlooking intelligent an hardwo...","Dallas, TX",0
48,1577029386699669512,JessieVasco,0,1,0,0,0,False,False,,,0


In [53]:
followers.to_csv("../../data/00-raw-data/followers.csv",index=False)

,user_id,screen_name,followers_count,friends_count,listed_count,favourites_count,tweet_num,protected,verfied,description,location,label
0,2198516225,_Banzi_,27,199,0,601,122,False,False,,Joburg,1
1,1504258025804210176,DelorbeTori,81,2771,0,0,6,False,False,"Tori , 22😏 , Submissive🌸",,1
2,1581474148571877377,omkarVyawahare2,0,56,0,24,0,False,False,,,1
3,2967681610,L1BERTE_S,50,68,1,8735,2238,False,False,"Les querelles ne dureraient pas longtemps, si ...",Provence-Alpes-Côte d'Azur,1
4,1400890434,mistamomo_,356,1104,19,18347,20160,False,False,Amana.....abin riko da hanun dama 🤝 #freepales...,N/a,1
...,...,...,...,...,...,...,...,...,...,...,...,...
45,1576506437823504389,FranciscoPalob3,0,2,0,0,0,False,False,,,0
46,1570254264496177153,Nkindianfoods22,1,21,0,2,6,False,False,I m Narinder kaur I live in fort worth tx I ma...,"Fort Worth, TX",0
47,1580761571457994754,Dearl88447683,14,274,0,3,1,False,False,"Single, gay, goodlooking intelligent an hardwo...","Dallas, TX",0
48,1577029386699669512,JessieVasco,0,1,0,0,0,False,False,,,0
